<div class="alert alert-block alert-success">  
<h1><center><strong>SPACESHIP TITANIC PROBLEM 🚀</strong></center></h1>

#### If this helped in your learning, then please **UPVOTE** – as they are the source of motivation!

## **Important Libraries** 📃

In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, RandomizedSearchCV,StratifiedKFold
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,recall_score
import xgboost as xgb
from sklearn.preprocessing import StandardScaler,LabelEncoder
import plotly 
import plotly.express as px

In [ ]:
train_data = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

In [ ]:
train_data.head()

In [ ]:
train_data.info()

In [ ]:
plt.figure(figsize=(8,6))
train_data.dtypes.value_counts().plot(kind='pie',autopct='%.1f%%',colors=['#264653', '#2a9d8f','#e9c46a'])

**So Most of the Data are Categorical and others 50% of data are from int,float and bool**

In [ ]:
plt.figure(figsize=(14,16))
graph = sns.displot(data=train_data.isna().melt(value_name='missing'), y='variable',hue='missing',aspect=2.0,multiple='fill',palette=['#264653', '#2a9d8f'])
graph.set(ylabel='Features')

In [ ]:
counts = train_data[train_data.columns].isnull().sum()
perc = round(counts / train_data.shape[0] * 100,2)

df = pd.DataFrame({'counts' : counts, 'percentage' : perc})
df

As we See that the Percentage as well as the Counts of Nan in every feature is not much high.
So, we do not drop any column on the basis of Nan but we fill these Nans using multiple ways.

In [ ]:
train_data.describe()

As we Clearly See that Except Age Column, Other Remaining Columns have outliers. We tackle these also. But first we check Categorical Data that if they have any Outliers like this.

In [ ]:
train_data.select_dtypes('object').apply(pd.Series.nunique,axis=0)

Some Categorical Column have low Cardinality which shows that they have approx zero Variance but some of the Columns Like: Name,PassengerId and Cabin have High Cardinality. we will check further more on these columns

In [ ]:
def flag_creation(df):
    nan_cols = [col for col in df.columns if df[col].isnull().sum() != 0]
    for c in nan_cols:
        if c!='Name' and c!='Cabin':
            df[c+'_flag'] = np.where(df[c].isna() == True,1,0)
        
    return df

train_data = flag_creation(train_data)
test_data = flag_creation(test_data)

In [ ]:
def fill_nan(df):
    for c in df.columns:
        if df[c].dtype == 'object':
            df[c] = df[c].fillna(df.groupby(c)['CryoSleep'].mean())
        elif df[c].dtype != 'object':
            df[c] = df[c].fillna(df[c].mode()[0])
                    
    return df

train_data = fill_nan(train_data)
test_data = fill_nan(test_data)

In [ ]:
cols = ['HomePlanet','Destination','VIP','CryoSleep']
def Encoding(df,cols):
    le = LabelEncoder()
    for c in cols:
        df[c] = le.fit_transform(df[c])
    return df

train_data = Encoding(train_data,cols)
test_data = Encoding(test_data,cols)

In [ ]:
def break_down_passengerId(df):
    df['group_id'] = df['PassengerId'].apply(lambda x: x.split('_')[0])
    
    return df

train_data = break_down_passengerId(train_data)
test_data = break_down_passengerId(test_data)

In [ ]:
def Cabin_Column(df):
    df['Cabin'] = df['Cabin'].fillna(df['Cabin'].mode()[0])  
    
    df['deck'] = df['Cabin'].apply(lambda x: x.split('/')[0])
    df['num'] = df['Cabin'].apply(lambda x: x.split('/')[1])
    df['side'] = df['Cabin'].apply(lambda x: x.split('/')[2])
    
    return df

train_data = Cabin_Column(train_data)
test_data = Cabin_Column(test_data)

In [ ]:
def plot(df,col):
    ax = px.histogram(df,x=col,color='Transported',width=500,height=300)
    ax.show()

In [ ]:
plot(train_data,'HomePlanet')

People from Europa Transported, but People of Mars and Earth are not Transported. This can be insightful.

In [ ]:
plot(train_data,'Destination')

Destination Named "TRAPPIST-1e"'s People are more Transported then other two Destinations.

In [ ]:
plot(train_data,'CryoSleep')

* The People who were not in CryoSleep were more Transported then Those who were in CryoSleep in Ward.
* So, we conclude that those who were confined in their cabins were not Transported most.

In [ ]:
plot(train_data,'VIP')

We Clearly see that People who did not Paid for VIP Services are likely to Transported. but why ? 
Maybe they have some reasons.

In [ ]:
plot(train_data,'deck')

Most of the People were From Deck G and F and Moslty Trasnported from these Decks. 

In [ ]:
plot(train_data,'side')

***Removing Outliers from a given threshold***

In [ ]:
train_data.drop(train_data[train_data['RoomService'] > 10000].index,axis=0,inplace=True)
train_data.drop(train_data[train_data['VRDeck'] > 20000].index,axis=0,inplace=True)
train_data.drop(train_data[train_data['Spa'] > 20000].index,axis=0,inplace=True)
train_data.drop(train_data[train_data['FoodCourt'] > 20000].index,axis=0,inplace=True)
train_data.drop(train_data[train_data['ShoppingMall'] > 20000].index,axis=0,inplace=True)

In [ ]:
new_train_data = train_data.drop(['Cabin','PassengerId','Name'],axis=1)
new_test_data = test_data.drop(['Cabin','PassengerId','Name'],axis=1)

In [ ]:
def featureCreation(df):
    
    le = LabelEncoder()
    df['deck'] = le.fit_transform(df['deck'])
    df['num'] = le.fit_transform(df['num'])
    df['side'] = le.fit_transform(df['side'])

    
    return df

new_train_data = featureCreation(new_train_data)
new_test_data  = featureCreation(new_test_data)

In [ ]:
def Extra(df):
    df['Total_Spend'] = df[['RoomService','Spa','FoodCourt','ShoppingMall','VRDeck']].sum(axis=1) 
    return df

new_train_data = Extra(new_train_data)
new_test_data = Extra(new_test_data)

In [ ]:
def check_family_members(df):
    df['No_of_family_members'] = df['group_id'].map(df['group_id'].value_counts())
    return df
new_train_data = check_family_members(new_train_data)
new_test_data = check_family_members(new_test_data)

In [ ]:
new_train_data.head(4)

In [ ]:
def Age(df):
    df['Youth'] = df['Age'].apply(lambda x: 1 if x>0 and x<=18 else 0)
    df['Adult'] = df['Age'].apply(lambda x: 1 if x>18 else 0)
    
    return df

new_train_data = Age(new_train_data)
new_test_data = Age(new_test_data)

In [ ]:
plt.figure(figsize=(24,26))
sns.heatmap(new_train_data.corr(),annot=True,cmap='twilight')

In [ ]:
def Scaling(df,name):
    
    if name == 'train':
        t_data = df.copy().drop('Transported',axis=1)
        ss = StandardScaler()
        ss.fit(t_data)
        dff = pd.DataFrame(ss.transform(t_data),index=t_data.index,columns=t_data.columns)
        dff['Transported'] = df['Transported']
    
    else:
        ss = StandardScaler()
        ss.fit(df)
        dff = pd.DataFrame(ss.transform(df),index=df.index,columns=df.columns)
        
    return dff

train_csv = Scaling(new_train_data,'train')
test_csv = Scaling(new_test_data,'test')

In [ ]:
X = train_csv.drop(['Transported','Age','group_id'],axis=1)
Y = train_csv.Transported
test_csv = test_csv.drop(['Age','group_id'],axis=1)

In [ ]:
accuracy = []
recall = []
fold_no =[]
def model(X,Y,folds):
    skf = StratifiedKFold(n_splits=3,shuffle=True,random_state=0)
    for fold,(train_idx,test_idx) in enumerate(skf.split(X,Y)):
        train_X,val_X = X.iloc[train_idx],X.iloc[test_idx]
        train_y,val_y = Y.iloc[train_idx],Y.iloc[test_idx]
        
        clf = RandomForestClassifier(n_estimators=100,max_depth=12)
        
        clf.fit(train_X,train_y)
        pred = clf.predict(val_X)
        
        accuracy.append(round(accuracy_score(val_y,pred),4))
        recall.append(round(recall_score(val_y,pred),4))
        fold_no.append(fold+1)
        
    
    
    result_df = pd.DataFrame({'Accuracy' : accuracy,'Recall' : recall},index=fold_no)
    
    return result_df,clf

In [ ]:
result_df,model = model(X,Y,5)
result_df

In [ ]:
predictions = model.predict(test_csv)
sub = pd.read_csv('../input/spaceship-titanic/sample_submission.csv')
t = pd.read_csv('../input/spaceship-titanic/test.csv')
sub['PassengerId'] = t['PassengerId']
sub['Transported'] = predictions
sub['Transported'] = sub['Transported'].astype(bool)
sub.to_csv('my_submission.csv',index=False)